In [11]:
!pwd

/home/yutankim/Desktop/EfficientDet_TensorFlow2


In [1]:
import tensorflow as tf

from configuration import Config
from core.efficientdet import EfficientDet, PostProcessing


def print_model_summary(network):
    sample_inputs = tf.random.normal(shape=(Config.batch_size, Config.get_image_size()[0], Config.get_image_size()[1], Config.image_channels))
    sample_outputs = network(sample_inputs, training=True)
    network.summary()

#모델 구조
model = EfficientDet()
print_model_summary(model)

#weight 불러오기
load_weights_from_epoch = Config.load_weights_from_epoch_quan
model.load_weights(filepath=Config.save_model_dir+"epoch-{}".format(load_weights_from_epoch))
#model.load_weights(filepath=Config.save_model_dir+"saved_model")

2021-10-25 18:54:11.933829: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-25 18:54:12.485360: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22854 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:3b:00.0, compute capability: 7.5
2021-10-25 18:54:13.632722: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2021-10-25 18:54:14.288179: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Model: "efficient_det"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficient_net (EfficientNet) multiple                  6771296   
_________________________________________________________________
bi_fpn (BiFPN)               multiple                  129126    
_________________________________________________________________
box_class_predict (BoxClassP multiple                  249069    
Total params: 7,149,491
Trainable params: 7,106,515
Non-trainable params: 42,976
_________________________________________________________________


In [28]:
dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'/*')
print(dataset_list)
image = next(iter(dataset_list))
image = tf.io.read_file(image)
image = tf.io.decode_jpeg(image, channels=3)
image = tf.image.resize(image, (512,512))
image = tf.expand_dims(image, 0)
print(image)
# 여기 부분, 기존 코드의 data_preprocesssing 과 동일하게 맞추기
def representative_data_gen():
    dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'/*')
    for i in range(100):
        image = next(iter(dataset_list))
        image = tf.io.read_file(image)
        image = tf.io.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, (512,512))
        # image = tf.cast(image / 255., tf.float32)
        image = tf.expand_dims(image, 0)
# Model has only one input so each data point has one element
        yield [image]

#     #### 수정좀 해보기
# def representative_data_gen():
#   dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')
#   for i in range(100):
#     image = next(iter(dataset_list))
#     image = tf.io.read_file(image)
#     image = tf.io.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, (512,512))
#     # image = tf.cast(image / 255., tf.float32)
#     image = tf.expand_dims(image, 0)
# # Model has only one input so each data point has one element
#     yield [image]

<ShuffleDataset shapes: (), types: tf.string>
tf.Tensor(
[[[[ 62.         54.         18.       ]
   [ 73.375      65.375      29.375    ]
   [ 97.75       89.75       53.75     ]
   ...
   [ 56.         53.         34.       ]
   [ 56.         53.         34.       ]
   [ 55.125      52.125      33.125    ]]

  [[ 89.52734    81.52734    44.621094 ]
   [112.91016   104.91016    68.00391  ]
   [ 99.90234    91.90234    54.996094 ]
   ...
   [ 56.         53.         34.       ]
   [ 56.         53.         34.       ]
   [ 55.91797    52.91797    33.91797  ]]

  [[ 66.42969    60.960938   19.742188 ]
   [ 76.81641    71.34766    32.34375  ]
   [ 58.88672    53.41797    14.4140625]
   ...
   [ 56.527344   53.527344   34.527344 ]
   [ 56.         53.         34.       ]
   [ 56.         53.         34.       ]]

  ...

  [[ 72.92969    64.30469    45.773438 ]
   [ 68.65625    60.03125    41.5      ]
   [ 66.546875   57.921875   39.390625 ]
   ...
   [ 54.878906   49.722656   29.53125  ]


In [29]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.experimental_new_quantizer = True
converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8
converter.inference_output_type = tf.float32

tflite_model_quant = converter.convert()

2021-10-25 19:10:09.068686: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2021-10-25 19:10:09.068823: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-10-25 19:10:09.070712: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22854 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:3b:00.0, compute capability: 7.5
2021-10-25 19:10:09.182659: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.017ms.
  function_optimizer: function_optimizer did nothing. time = 0.001ms.

2021-10-25 19:10:12.039505: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:351] Ignored output_format.
2021-10-25 19:10:12.039552: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuf

RuntimeError: tensorflow/lite/kernels/reshape.cc:69 num_input_elements != num_output_elements (576 != 512)Node number 503 (RESHAPE) failed to prepare.


In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
for item in interpreter.get_tensor_details():
    for key in item.keys():
        print("%s : %s" % (key, item[key]))
    print("")

In [ ]:
with open('./tflite_model_quant.tflite', 'wb') as f:
  f.write(tflite_model_quant)

In [ ]:
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
print(input_details)
print(output_details)

In [ ]:
model_path = "C:/Users/yutan/Desktop/EfficientDet_TensorFlow2/tflite_model_quant.tflite"

def load_model(path):  
    interpreter = tf.lite.Interpreter(model_path = path)
    return interpreter

def prediction(model, data):
    interpreter = model
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    input_data = np.array(data, dtype = np.uint8)
    interpreter.set_tensor(input_details['index'], input_data)
    interpreter.invoke()
    output= interpreter.get_tensor(output_details['index'])
    return output

In [ ]:
dataset_list = tf.data.Dataset.list_files('./data/fire_smoke/train/JPEGImages'+'\\*')

image = next(iter(dataset_list))
image = tf.io.read_file(image)
image = tf.io.decode_jpeg(image, channels=3)
image = tf.image.resize(image, (512,512))
# image = tf.cast(image / 255., tf.float32)
image = tf.expand_dims(image, 0)
print(image.shape)

In [ ]:
import numpy as np

model = load_model(model_path)
output = prediction(model, image)

In [ ]:
print(output.shape)
print(output)